# plot

> A module to plot the results of the benchmark

In [ ]:
#| default_exp plot

In [ ]:
#| export
import plotly.graph_objects as go
from fasterbench.utils import *

In [ ]:
#| export
def create_radar_plot(benchmark_results):
    # Define metrics with icons, hover text format, and units
    metrics = {
        '💾': {  # Storage icon
            'value': benchmark_results['disk_size'] / 1e6,
            'hover_format': 'Model Size: {:.2f} MB',
            'unit': 'MB'
        },
        '🧮': {  # Calculator icon for parameters
            'value': parse_metric_value(benchmark_results['num_parameters']),
            'hover_format': 'Parameters: {:.2f}M',
            'unit': 'M'
        },
        '⏱️': {  # Clock icon for latency
            'value': benchmark_results['cpu_latency'],
            'hover_format': 'Latency: {:.2f} ms',
            'unit': 'ms'
        },
        '⚡': {  # Lightning bolt for MACs
            'value': parse_metric_value(benchmark_results['macs']),
            'hover_format': 'MACs: {:.2f}G',
            'unit': 'G'
        },
        '🔋': {  # Battery icon for energy
            'value': benchmark_results['avg_energy'] * 1e6,
            'hover_format': 'Energy: {:.3f} mWh',
            'unit': 'mWh'
        }
    }
    
    # Find min and max values for each metric
    reference_values = {
        '💾': {'min': 0, 'max': max(metrics['💾']['value'], 500)},    # Model size (MB)
        '🧮': {'min': 0, 'max': max(metrics['🧮']['value'], 50)},     # Parameters (M)
        '⏱️': {'min': 0, 'max': max(metrics['⏱️']['value'], 50)},    # Latency (ms)
        '⚡': {'min': 0, 'max': max(metrics['⚡']['value'], 1000)},      # MACs (G)
        '🔋': {'min': 0, 'max': max(metrics['🔋']['value'], 5)}       # Energy (mWh)
    }
    
    # Normalize values and create hover text
    normalized_values = []
    hover_texts = []
    labels = []
    
    for icon, metric in metrics.items():
        # Min-max normalization
        normalized_value = (metric['value'] - reference_values[icon]['min']) / \
                         (reference_values[icon]['max'] - reference_values[icon]['min'])
        normalized_values.append(normalized_value)
        
        # Create hover text with actual value
        hover_texts.append(metric['hover_format'].format(metric['value']))
        labels.append(icon)
    
    # Add first values again to close the polygon
    normalized_values.append(normalized_values[0])
    hover_texts.append(hover_texts[0])
    labels.append(labels[0])
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatterpolar(
        r=normalized_values,
        theta=labels,
        fill='toself',
        name='Model Metrics',
        hovertext=hover_texts,
        hoverinfo='text',
        line=dict(color='#FF8C00'),  # Bright orange color
        fillcolor='rgba(255, 140, 0, 0.3)'  # Semi-transparent orange
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1],
                showticklabels=False,  # Hide radial axis labels
                gridcolor='rgba(128, 128, 128, 0.5)',  # Semi-transparent grey grid lines
                linecolor='rgba(128, 128, 128, 0.5)'   # Semi-transparent grey axis lines
            ),
            angularaxis=dict(
                tickfont=dict(size=24),  # Icon labels
                gridcolor='rgba(128, 128, 128, 0.5)'  # Semi-transparent grey grid lines
            ),
            bgcolor='rgba(0,0,0,0)'  # Transparent background
        ),
        showlegend=False,

        margin=dict(t=100, b=100, l=100, r=100),
        paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
        plot_bgcolor='rgba(0,0,0,0)'    # Transparent background
    )
    
    return fig